# 数据处理

## Jina ColBert v2

In [4]:
!export HF_ENDPOINT=https://hf-mirror.com

In [ ]:
from ragatouille import RAGPretrainedModel
model_path = "jinaai/jina-colbert-v2"
model_path = "/root/xiatian/models/jina-colbert-v2"
RAG = RAGPretrainedModel.from_pretrained(model_path)
docs = [
    "ColBERT is a novel ranking model that adapts deep LMs for efficient retrieval.",
    "Jina-ColBERT is a ColBERT-style model but based on JinaBERT so it can support both 8k context length, fast and accurate retrieval.",
]
RAG.index(docs, index_name="demo")
query = "What does ColBERT do?"
results = RAG.search(query)

## Transformers运行训练完成的模型

In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch

model_path = "/root/xiatian/LLaMA-Factory/models/llama3_tibkat_lora_sft"
tokenizer = AutoTokenizer.from_pretrained(model_path)
dtype = torch.float16
device = "auto"
model = LlamaForCausalLM.from_pretrained(model_path, torch_dtype=dtype, device_map=device)


/root/.cache/pypoetry/virtualenvs/llms4subjects-s_Ootod2-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 9/9 [00:08<00:00,  1.10it/s]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [2]:
print(tokenizer.eos_token_id)
print(model.device)

128001
cuda:2


In [3]:
from llms4subjects.instance import load_jsonline_file
from llms4subjects.prompt import make_input_text

eval_ds = load_jsonline_file("./db/instance/merged/dev.jsonline")

100%|██████████| 19949/19949 [00:01<00:00, 13565.43it/s]


In [7]:
doc = eval_ds[3]
input_text = make_input_text(doc["title"], doc["abstract"])

device = model.device
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
max_length = len(input_ids[0])+1000
attention_mask = torch.ones_like(input_ids)
with torch.no_grad():
    output = model.generate(input_ids, 
                            attention_mask=attention_mask,
                            max_length=max_length,
                            repetition_penalty=1.2,
                            pad_token_id=tokenizer.eos_token_id,
                            num_return_sequences=1,
                            do_sample=False)
out = tokenizer.decode(output[0], skip_special_tokens=True)
print(input_text)
print("==========")
out_text = out[len(input_text):].strip()
print(out_text)

/root/.cache/pypoetry/virtualenvs/llms4subjects-s_Ootod2-py3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/llms4subjects-s_Ootod2-py3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


You are a librarian responsible for assigning a set of subject tags to a technical document based on its title and abstract. Based on the information provided below, please output the related subjects, with one subject per line.
- Title: Sicherung des Familieneinflusses in Familienunternehmen : Symposium der Forschungsstelle für Familienunternehmen der Universität Bayreuth am 6./7. Oktober 2016
- Abstract: Der Tagungsband dokumentiert den Stand der wissenschaftlichen Forschung zum Einflusses der Familie in Familienunternehmen und dient zugleich dem Wissenstransfer zwischen Wissenschaft und Praxis. Dazu werden verschiedene Instrumente der Sicherung des Einflusses der Familie in Familienunternehmen von namhaften Experten unter verschiedenen Blickwinkeln analysiert und bewertet. Dies gilt etwa für die Sicherung des Familieneinflusses durch Beiräte, Stiftungen, Interim-Management, notarielle Verträge und gemeinsame Werte. Neben dieser nationalen Betrachtung enthält der Band auch eine vergl

In [6]:
len(input_ids[0])

396

## 观察验证机和训练集文件是否有交集

In [6]:
from pathlib import Path

train_path = Path("./data/shared-task-datasets/TIBKAT/tib-core-subjects/data/train/")
dev_path = Path("./data/shared-task-datasets/TIBKAT/tib-core-subjects/data/dev/")
core_files = list(train_path.glob("**/*.jsonld"))
core_file_names = [f.name for f in core_files]
print(len(core_file_names))
print(len(set(core_file_names)))

41902
41902


In [7]:
train_path = Path("./data/shared-task-datasets/TIBKAT/all-subjects/data/train/")
dev_path = Path("./data/shared-task-datasets/TIBKAT/all-subjects/data/dev/")
all_files = list(train_path.glob("**/*.jsonld"))
all_file_names = [f.name for f in all_files]
all_dict = {f.name: f for f in all_files}
print(len(all_file_names))
print(len(set(all_file_names)))

81937
81937


In [8]:
missed = []
myset = set(all_file_names)
for f in core_files:
    if f.name not in myset:
        missed.append(f)
    elif f.stat().st_size != all_dict[f.name].stat().st_size:
        print("ERROR: ", f)
        
print(len(missed))
missed_one = missed[0]
target = None
for f, name in zip(core_files, core_file_names):
    if missed_one == name:
        target = f
        break
target


16815


In [9]:
import hashlib
def md5_of_file(file_path) -> str:
    # 创建一个md5对象
    md5_hash = hashlib.md5()

    # 以二进制模式打开文件，读取文件内容并更新到md5对象中
    with open(file_path, "rb") as file:
        # 读取文件内容，每次读取1024字节（你可以根据需要调整这个值）
        for chunk in iter(lambda: file.read(1024), b""):
            md5_hash.update(chunk)

    # 获取十六进制格式的md5值
    md5_value = md5_hash.hexdigest()
    return md5_value

In [11]:
train_path = Path("./data/shared-task-datasets/TIBKAT/merged-subjects/data/train/")
dev_path = Path("./data/shared-task-datasets/TIBKAT/merged-subjects/data/dev/")
merged_files = list(train_path.glob("**/*.jsonld"))
merged_file_names = [f.name for f in merged_files]
merged_dict = {}
print(len(merged_file_names))
print(len(set(merged_file_names)))

n_duplicate = 0
for f in merged_files:
    if f.name not in merged_dict:
        merged_dict[f.name] = f
    else:
        existed_file = merged_dict[f.name]
        m1 = md5_of_file(f.as_posix())
        m2 = md5_of_file(existed_file.as_posix())
        n_duplicate += 1
        if m1 == m2:
            print(f"same:\n\t{f.as_posix()}\n\t{existed_file.as_posix()}")
        else:
            print(f"nonsame:\n\t{f.as_posix()}\n\t{existed_file.as_posix()}")
            
print("duplicate num: ", n_duplicate)

98801
98752
same:
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/en/3A1702658414.jsonld
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/de/3A1702658414.jsonld
same:
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/en/3A1625006101.jsonld
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/de/3A1625006101.jsonld
same:
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/en/3A877616914.jsonld
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/de/3A877616914.jsonld
same:
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/en/3A480350477.jsonld
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/de/3A480350477.jsonld
same:
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/en/3A1773269402.jsonld
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/de/3A1773269402.jsonld
same:
	data/shared-task-datasets/TIBKAT/merged-subjects/data/train/Book/

In [13]:
from pathlib import Path
merged_folder = "./data/shared-task-datasets/TIBKAT/merged-subjects/data"

merged_de_files = list(Path(merged_folder).glob("**/de/*.jsonld"))

n_cleaned = 0
for f in merged_de_files:
    en_dir = Path(f.parent.parent, "en")
    en_file = Path(en_dir, f.name)
    if en_file.exists():
        en_file.unlink()
        n_cleaned += 1
print("cleaned: ", n_cleaned)

cleaned:  0


In [10]:
from pathlib import Path
train_path = Path("./data/shared-task-datasets/TIBKAT/merged-subjects/data/train/")
dev_path = Path("./data/shared-task-datasets/TIBKAT/merged-subjects/data/dev2/")
train_files = list(train_path.glob("**/*.jsonld"))
train_file_names = [f.name for f in train_files]
train_file_names = set(train_file_names)

dev_files = list(dev_path.glob("**/*.jsonld"))
duplicated = 0
for f in dev_files:
    if f.name in train_file_names:
        duplicated += 1
        f.unlink()
        
print("dev num: ", len(dev_files))
print(f"duplicated: {duplicated}")



dev num:  11649
duplicated: 0


## 根据主题嵌入相似度进行推荐

In [ ]:
print("HELLO")

In [5]:
print("________________\n by subject embedding query\n---------------")
pred_codes, pred_names = by_similar_subjects(title, abstract, "core", 10)

print("Title:\t", title)
print("Abstract:\t", abstract)
print("Ground Truth:")
for code, name in zip(true_codes, true_names):
    print(f"\t\t{code} -> {name}")
    
print("Predicted:")
for code, name in zip(pred_codes, pred_names):
    print(f"\t\t{code} -> {name}")
    
print("Matched Result:")
for code, name in zip(true_codes, true_names):
    if code in pred_codes:
        print(f"\t\t{code} -> {name}: \tSUCCESS")
    else:
        print(f"\t\t{code} -> {name}: \tFAILURE")
        
subject_name = "Methodik"
subject_eq = SubjectEmbeddingQuery("./db/subject/all/")
mycode = subject_db.get_code_by_name(subject_name)
print(subject_name, mycode)
namecodes = subject_eq.get_namecodes_by_name(subject_name, 3)
print(namecodes)

________________
 by subject embedding query
---------------
Title:	 "Städte für alle" - über visionären und machbaren Städtebau : Martin Neuffer und Rudolf Koldewey ; Dokumentation des Symposiums am 20.06.2005 in Hannover
Abstract:	 Zur Würdigung der beiden Oberstadtdirektoren Martin Neuffer und Rudolf Koldewey der Landeshauptstadt Hannover fand im Juni 2005 das Symposium statt. Die visionären Vorstellungen und Entwicklungsmöglichkeiten Martin Neuffers ("Städte  für alle- Entwurf einer Städtepolitik" als programmatischer Titel eines seiner Bücher) und die eher nüchterne und pragmatische Denk- und Handlungsweise Rudolf Koldeweys stellten das gedankliche Spannungsfeld dar, in dem die Einzelbeiträge dieser Dokumentation angesiedelt sind. Im Ergebnis werden die aktuellen Herausforderungen an Städtebau und Stadtentwicklung vor dem Hintergrund zum Teil gravierender Veränderungen der stadtentwicklungspolitischen Rahmenbedingungen aus Sicht der Kommunalpolitik, der Stadtforschung, der planend

# LLM TEST

In [1]:
from llms4subjects.llm import LLM

bot = LLM(
    base_url="http://14.152.45.76:3073/v1",
    model="llama3.3:latest",
)

In [15]:
import json

text = """Aufgrund der steigenden Einspeisung elektrischer Leistung durch erneuerbare Energieanlagen und der gleichzeitig voranschreitenden schrittweisen Abschaltung von konventionellen Kohlekraftwerken, steigen die Anforderungen an eine zuverl\u00e4ssige, kosteng\u00fcnstige und klimafreundliche Bereitstellung von Flexibilit\u00e4ten zum Erhalt der Systemstabilit\u00e4t. Mit der Gesamtsystembetrachtung aus Strom- und Gassektor k\u00f6nnen durch die Kopplung neue Freiheitsgrade erschlossen werden. In diesem Beitrag wird vorgestellt, wie eine sektoren\u00fcbergreifende Erbringung von Systemdienstleistungen vom Strom- zum Gasnetz in einem dynamischen Energiesystemmodell im Zeitbereich der Mittelzeitdynamik abgebildet werden kann. Anhand von numerischen Fallstudien wird am Beispiel von Th\u00fcringen f\u00fcr verschiedene Szenarien ausgewertet, inwiefern durch eine sektoren\u00fcbergreifende Betrachtung Flexibilit\u00e4tspotenziale realisiert und die Betriebsgrenzen und somit die Stabilit\u00e4tsbedingungen beider Sektoren eingehalten werden k\u00f6nnen.",
            "contributor": "Technische Informationsbibliothek (TIB)"""
text = "hello world, summer."
response = bot.complete(f"Please enter the text below for translation into German. If the text is already in German, then output it directly. Output only the translation result without any other auxiliary information.\n\n{text}", max_tokens=2048)
response = json.loads(response)
response["choices"][0]["text"].strip()

'Hallo Welt, Sommer.'

In [9]:
with open('TIBKAT-core.jsonline', "r", encoding="utf-8") as f:
    items = [json.loads(line) for line in f.readlines()]
        

In [11]:
e = items[0]
e['title_DE'] = 'hello'
e['title_EN'] = 'hello2'


In [12]:
items[0]

{'id': '3A271864087',
 'title': 'Evaluation zur Verbesserung der Qualität der Lehre und weitere Maßnahmen',
 'abstract': 'Die 22. AHD-Jahrestagung umfasste Kurzreferate, Werkstattseminare und Arbeitsgruppen zu weiteren wesentlichen Massnahmen: "Hochschuldidaktische Weiterbildung", "Studienzeitverkuerzung", "Lehrberichte", "Materielle und immaterielle Anreize", "Reformstudiengaenge", "Kreatives Schreiben", "Fachuebergreifendes Lehren und Lernen" und "Tutorenprogramme", "Verbesserung der Qualitaet der Lehre durch unterschiedliche Massnahmen am Beispiel eines TEMPUS-Projekts zwischen der Universitaet Ljubljana und der FU Berlin, Arbeitsstelle Hochschuldidaktik" und schliesslich "Vorschlaege des BMBW zur Verbesserung der Qualitaet der Lehre". Gliederung: I. Eroeffnung und Einfuehrung (Gerlach, Johann Wilhelm - Webler, Wolff- Dietrich - Friedrich, Hans Rainer - Thies, Erich - Berendt, Brigitte) II. - 1. Eroeffnungsreferat zur Podiumsdiskussion: Webler, Wolff-Dietrich: "Evaluation als geeign

In [1]:
import json
from tqdm import tqdm
import requests
import re
import faiss
from pathlib import Path
import numpy as np

dim = 1024
index = faiss.IndexFlatIP(dim)
with open("./embedding-core.txt", "r", encoding="utf-8") as f_in:
    for line in tqdm(f_in.readlines()):
        parts = re.split(r"[,\t]", line)
        value = [float(v) for v in parts[1:]]
        value = np.array(value, dtype=np.float32).reshape(1, dim)
        index.add(value)

100%|██████████| 41902/41902 [00:28<00:00, 1474.93it/s]


In [1]:
from llms4subjects.instance import instance_db_merged

n = instance_db_merged.num()
print(n)
instance_db_merged.to_alpaca("./db/sft/tibkat-all.json")
instance_db_merged.to_alpaca("/root/xiatian/LLaMA-Factory/data/tibkat/tibkat-all.json")


98752


100%|██████████| 98752/98752 [00:05<00:00, 19240.39it/s]


In [ ]:
from llms4subjects.instance import merge_all_to_alpaca

# 包含了merged目录下的所有jsonld文件，计划在最终验证时训练该数据
merge_all_to_alpaca("./db/sft/tibkat-all-2.json")


100%|██████████| 118701/118701 [00:00<00:00, 612033.80it/s]
